# Flash Drum and Distillation Column

Simulating two fundamental separation processes: an isothermal binary flash drum and a multi-tray distillation column built from individual `DistillationTray` blocks wired in series.

## Part 1: Isothermal Flash Drum

A flash drum separates a liquid feed into vapor and liquid streams using vapor-liquid equilibrium (VLE). The drum uses Raoult's law with Antoine correlations to compute K-values:

$$K_i = \frac{P^\text{sat}_i(T)}{P}$$

The Rachford-Rice equation determines the vapor fraction $\beta$, from which the vapor ($y_i$) and liquid ($x_i$) compositions follow.

In [ ]:
import matplotlib.pyplot as plt

from pathsim import Simulation, Connection
from pathsim.blocks import Source, Scope

from pathsim_chem.process import FlashDrum, DistillationTray

Configure a flash drum for a benzene-toluene mixture (default Antoine coefficients). Feed an equimolar mixture at 1 atm and sweep temperature from 340 K to 400 K. This range covers the bubble point (~365 K) and dew point (~380 K) of the mixture, so we can observe the transition from all-liquid to two-phase to all-vapor.

In [ ]:
flash = FlashDrum(holdup=100.0)  # default benzene/toluene Antoine params

# Feed: 10 mol/s, equimolar (z1 = 0.5), 1 atm
F_src = Source(func=lambda t: 10.0)
z_src = Source(func=lambda t: 0.5)
T_src = Source(func=lambda t: 340.0 + t * 0.5)  # ramp 340 -> 400 K
P_src = Source(func=lambda t: 101325.0)

scp = Scope(labels=["V_rate", "L_rate", "y_1 (benzene)", "x_1 (benzene)"])

sim = Simulation(
    blocks=[F_src, z_src, T_src, P_src, flash, scp],
    connections=[
        Connection(F_src, flash),           # F -> port 0
        Connection(z_src, flash[1]),         # z_1 -> port 1
        Connection(T_src, flash[2]),         # T -> port 2
        Connection(P_src, flash[3]),         # P -> port 3
        Connection(flash, scp),              # V_rate -> scope 0
        Connection(flash[1], scp[1]),        # L_rate -> scope 1
        Connection(flash[2], scp[2]),        # y_1 -> scope 2
        Connection(flash[3], scp[3]),        # x_1 -> scope 3
    ],
    dt=0.5,
)

sim.run(120)

In [ ]:
time, signals = scp.read()
T_sweep = 340.0 + time * 0.5

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(T_sweep, signals[0], label="Vapor rate")
ax1.plot(T_sweep, signals[1], label="Liquid rate")
ax1.set_xlabel("Temperature [K]")
ax1.set_ylabel("Flow rate [mol/s]")
ax1.set_title("Flash Drum Flow Rates")
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(T_sweep, signals[2], label=r"$y_1$ (vapor)")
ax2.plot(T_sweep, signals[3], label=r"$x_1$ (liquid)")
ax2.axhline(0.5, color="gray", ls="--", alpha=0.4, label="Feed")
ax2.set_xlabel("Temperature [K]")
ax2.set_ylabel("Mole fraction (benzene)")
ax2.set_title("VLE Compositions")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

As temperature increases, more liquid vaporizes (higher vapor rate). The vapor is enriched in the lighter component (benzene), while the liquid becomes richer in toluene — the classic VLE separation.

## Part 2: Distillation Column (5 Trays)

A distillation column is built by wiring multiple `DistillationTray` blocks in series. Each tray enforces vapor-liquid equilibrium with a constant relative volatility $\alpha$:

$$y = \frac{\alpha\, x}{1 + (\alpha - 1)\, x}$$

Under constant molar overflow (CMO), liquid flows down and vapor flows up with constant rates $L$ and $V$. We feed the column at the middle tray.

In [ ]:
# Column parameters
N_trays = 5
alpha = 2.5       # relative volatility
L = 5.0           # liquid flow rate [mol/s]
V = 5.0           # vapor flow rate [mol/s]
x_feed = 0.5      # feed composition (light component)

# Create tray blocks (all start at x = 0.5)
trays = [DistillationTray(M=1.0, alpha=alpha, x0=0.5) for _ in range(N_trays)]

# Liquid feed from condenser (enters tray 0 from above)
L_src = Source(func=lambda t: L)
x_top = Source(func=lambda t: 0.95)   # reflux composition (nearly pure light)

# Vapor feed from reboiler (enters tray N-1 from below)
V_src = Source(func=lambda t: V)
y_bot = Source(func=lambda t: 0.05)   # reboiler vapor (nearly pure heavy)

# Record composition on each tray
scp = Scope(labels=[f"Tray {i+1}" for i in range(N_trays)])

Wire the trays: liquid cascades downward (tray $i$ liquid output $\to$ tray $i+1$ liquid input), vapor rises upward (tray $i$ vapor output $\to$ tray $i-1$ vapor input). External feeds enter the top and bottom.

In [ ]:
connections = []

# Top tray (0): liquid from reflux
connections.append(Connection(L_src, trays[0]))        # L_in -> port 0
connections.append(Connection(x_top, trays[0][1]))     # x_in -> port 1

# Bottom tray (N-1): vapor from reboiler
connections.append(Connection(V_src, trays[-1][2]))    # V_in -> port 2
connections.append(Connection(y_bot, trays[-1][3]))    # y_in -> port 3

# Inter-tray connections
for i in range(N_trays - 1):
    # Liquid flows down: tray i -> tray i+1
    connections.append(Connection(trays[i], trays[i+1]))       # L_out -> L_in
    connections.append(Connection(trays[i][1], trays[i+1][1])) # x_out -> x_in

    # Vapor flows up: tray i+1 -> tray i
    connections.append(Connection(trays[i+1][2], trays[i][2])) # V_out -> V_in
    connections.append(Connection(trays[i+1][3], trays[i][3])) # y_out -> y_in

# Connect each tray's liquid composition to scope
for i, tray in enumerate(trays):
    connections.append(Connection(tray[1], scp[i]))  # x_out -> scope

sim = Simulation(
    blocks=[L_src, x_top, V_src, y_bot, *trays, scp],
    connections=connections,
    dt=0.05,
)

sim.run(30)

In [ ]:
time, signals = scp.read()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Dynamic tray composition evolution
for i in range(N_trays):
    ax1.plot(time, signals[i], label=f"Tray {i+1}")
ax1.set_xlabel("Time [s]")
ax1.set_ylabel(r"$x$ (light component)")
ax1.set_title("Tray Compositions Over Time")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Steady-state composition profile
x_profile = [tray.engine.state[0] for tray in trays]
tray_nums = list(range(1, N_trays + 1))

ax2.plot(tray_nums, x_profile, "o-", markersize=8)
ax2.set_xlabel("Tray number (top to bottom)")
ax2.set_ylabel(r"$x$ (light component)")
ax2.set_title("Composition Profile (Steady State)")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

The column separates the light and heavy components across its trays. The top tray is enriched in the light component (high $x$) while the bottom tray is depleted. The steady-state composition profile shows the characteristic staircase that a McCabe-Thiele diagram would predict for this relative volatility.